Vimos como adaptar um conjunto de dados por alguns modelos específicos.
De uma maneira geral, podemos considerar um modelo da forma
$$ f(x) = \alpha_1 \phi_1(x) + \dots + \alpha_n \phi_n(x), $$
e fazer o mesmo procedimento.

Esse problema com esse modelo é dito **quadrados mínimos lineares discretos.**

Para encontrar os parâmetros $\alpha$ fazemos o mesmo procedimento anterior:
$$ \min E(\alpha) = \frac{1}{2} \sum_{i=1}^m [f(x_i) - y_i]^2
= \frac{1}{2} \sum_{i=1}^m \bigg[\sum_{j=1}^n \alpha_j\phi_j(x_i) - y_i\bigg]^2. $$

Derivando em cada $\alpha_k$ nos dá um sistema $M\alpha = c$, onde
$$ M_{kj} = \sum_{i=1}^m \phi_k(x_i)\phi_j(x_i) \qquad \mbox{e}
\qquad c_j = \sum_{i=1}^m \phi_j(x_i) y_i. $$

Dados a função $\phi_j$ e os pontos $x_i$, podemos definir um vetor $\overline{\phi}_j = (\phi_j(x_1), \dots, \phi_j(x_m))^T$, isto é, o vetor obtido aplicando $\phi_j$ em cada elemento $x_i$.
Com essa definição, podemos escrever
$$ M_{k,j} = \langle\overline{\phi}_k,\overline{\phi}_j\rangle \qquad \mbox{e} \qquad
c_j = \langle\overline{\phi}_j, y\rangle. $$

O sistema $M\alpha = c$ é chamado de **sistema normal**, e ele tem uma característica muito importante:
A matriz $M$ é semidefinida positiva, e em muitos casos, definida positiva.

**Def.: ** Uma matriz simétrica $A$ é dita definida positiva se $v^TAv > 0$ para todo $v\neq 0$.

**Exercício: ** Demonstre que uma matriz simétrica definida positiva é não singular usando apenas a definição acima.

**Def.: ** Uma matriz simétrica $A$ é dita semi-definida positiva se $v^TAv \geq 0$ para todo $v$.

**Teo.: ** A é definida positiva se, e somente se, seus autovalores são positivos.

**Teo.: ** A é semi-definida positiva se, e somente se, seus autovalores são positivos ou nulos.

**Teo.: ** A é definida positiva se todos os determinantes $det(A[1:k,1:k])$ são positivos, para $k = 1,\dots,n$.

Existem muitos outros teoremas sobre definida positiva, mas um ponto importante aqui é que se $A$ é definida positiva, então ela é não-singular. Além disso, pelo teorema dos autovalores, sabemos como determinar se $A$ é definida positiva na prática.
Na prática, no entanto, as contas não valem a pena, e o teorema abaixo é muito mais útil:

**Teo.: ** $A$ é simétrica definida positiva se, e somente se, existe uma matriz $G$ quadrada não-singular tal que $A = GG^T$.

Podem existir mais de uma matriz $G$ que satisfaça essa propriedade, mas estamos preocupados com uma em particular:
A matriz da **decomposição de Cholesky**.

A decomposição de Cholesky de uma matriz $A$ simétrica definida positiva é a matriz $G$ triangular inferior que satisfaz $A = GG^T$
e com diagonal positiva.

In [2]:
G = [2 0; -1/2 sqrt(15)/2]

2x2 Array{Float64,2}:
  2.0  0.0    
 -0.5  1.93649

In [3]:
G*G'

2x2 Array{Float64,2}:
  4.0  -1.0
 -1.0   4.0

*Desenvolvimento feito em sala*

In [12]:
# Implementação
function cholesky(A)
    return chol(A)'
end

cholesky (generic function with 1 method)

In [13]:
A = [4.0  -1.0; -1.0  4.0]
cholesky(A)

2x2 LowerTriangular{Float64,Array{Float64,2}}:
  2.0  0.0    
 -0.5  1.93649

In [26]:
A = rand(10,10)
A = A*A' + eye(10)
G = cholesky(A)
norm(A - G*G', Inf)

2.6645352591003757e-15

## Forma matricial

Com os vetores $\overline{\phi}_1, \dots, \overline{\phi}$, podemos montar a matriz
$$ A = [\overline{\phi}_1 \cdots \overline{\phi}_n]. $$
Com essa matriz, podemos ver que $ A^TA = M$ e que $A^Ty = c$.
Ou seja, o sistema que queremos resolver é, na verdade
$$ A^TA\alpha = A^Tb, $$
ou seja, $A^T(A\alpha - b) = 0$.

Isso quer dizer que o sistema normal tem alguma relação com o sistema sobre-determinado $A\alpha = b$.
De fato, se pensarmos novamente no problema de minimizar aquela $E(\alpha)$, cada termo dentro do somatório é
uma linha de $A\alpha - b$.
Em outras palavras,
$$ E(\alpha) = \frac{1}{2}\sum_{i=1}^m (A\alpha-b)_i^2
= \frac{1}{2}\Vert A\alpha - b\Vert^2. $$

De fato, se calculamos $\nabla E(\alpha) = 0$, obtemos as equações normais.
Isso generaliza o problema de quadrados mínimos.
Esse problema aparece com frequência em outros contextos, e é bastante importante sabermos resolvê-lo eficientemente.

** Exercício: ** Calcule as derivadas parciais de $f(x) = \frac{1}{2}x^TQx + x^Tg$.
Escreva a forma do gradiente.

Computacionalmente, o problema de quadrados mínimos, como descrito acima, envolve o cálculo de $A^TA$, depois de Cholesky, e depois da resolução do sistema. Alguns códigos, em particular o [CHOLMOD](http://faculty.cse.tamu.edu/davis/suitesparse.html), calculam a decomposição de Cholesky diretamente da matriz $A$, o que reduz bastante o trabalho computacional.

Uma outra possibilidade de resolver esse problema seria usar algum método iterativo. Os métodos que vimos não são bons para isso, pois envolvem a formação da matriz $A$. No entanto, existe um outro método, chamado **Método dos Gradientes Conjugados**, que é bastante útil para esse problema.

In [46]:
A = rand(5000, 5);

In [47]:
v = rand(5)

5-element Array{Float64,1}:
 0.730023
 0.927737
 0.478004
 0.74684 
 0.877347

In [30]:
# Pra fazer A'*A*v tem duas opções:

In [87]:
@time (A'*A) * v

  0.000099 seconds (8 allocations: 656 bytes)


5-element Array{Float64,1}:
 4943.36
 5018.93
 4884.24
 4972.31
 5052.14

In [78]:
@time A' * (A * v)

  0.000078 seconds (10 allocations: 39.438 KB)


5-element Array{Float64,1}:
 4943.36
 5018.93
 4884.24
 4972.31
 5052.14

In [118]:
function quadmin(F, x, y)
    n = length(F)
    m = length(x)
    A = zeros(m, n)
    for j = 1:n
        f = F[j]
        for i = 1:m
            A[i,j] = f(x[i])
        end
    end
    return (A'*A)\(A'*y)
end

quadmin (generic function with 1 method)

In [130]:
using Plots
gr()

m = 500
x = sort(5 * rand(m) - 2) # x ∈ [-2,3]
y = 2 + 0.2 * exp(x) - 0.5 * x + 0.3 * x.^2 + rand(m)*0.2

scatter(x, y, ms=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [135]:
# O modelo é dado pelas funções
# f1(x) = 1, f2(x) = exp(x), f3(x) = x, f4(x) = x^2
F = [x->1, x->x, x->x^2, x->exp(x)]
α = quadmin(F, x, y)

scatter(x, y, ms=3)
plot!(t->dot(α, [f(t) for f in F]), x[1], x[end], c=:red, lw=2)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 51.3,221.322 56.6423,226.889 61.9847,232.349 67.3271,237.703 72.6695,242.949 78.0119,248.089 83.3543,253.12 88.6967,258.044 94.039,262.86 99.3814,267.567 
 104.724,272.165 110.066,276.654 115.409,281.033 120.751,285.302 126.093,289.46 131.436,293.508 136.778,297.444 142.121,301.268 147.463,304.979 152.805,308.577 
 158.148,312.062 163.49,315.432 168.832,318.687 174.175,321.826 179.517,324.849 184.86,327.754 190.202,330.542 195.544,333.21 200.887,335.759 206.229,338.187 
 211.572,340.493 216.914,342.676 222.256,344.736 227.599,346.67 232.941,348.479 238.283,350.159 243.626,351.711 248.968,353.132 254.311,354.422 259.653,355.578 
 264.995,356.6 270.338,357.484 275.68,358.23 281.023,358.836 286.365,359.299 291.707,359.617 297.05,359.789 302.392,359.811 307.734,359.682 313.077,359.399 
 318.419,358.959 323.762,358.359 329.104,357.596 334.446,356.667 339.789,355.569 345.131,354.298 350.474,352.851 355.816,351.223 361.158,349.411 366.501,347.411 
 371.843,345.217 377.185,342.825 382.528,340.23 387.87,337.427 393.213,334.411 398.555,331.175 403.897,327.714 409.24,324.021 414.582,320.089 419.925,315.911 
 425.267,311.48 430.609,306.788 435.952,301.827 441.294,296.588 446.636,291.063 451.979,285.24 457.321,279.111 462.664,272.664 468.006,265.889 473.348,258.774 
 478.691,251.307 484.033,243.474 489.376,235.262 494.718,226.657 500.06,217.644 505.403,208.206 510.745,198.327 516.087,187.99 521.43,177.175 526.772,165.865 
 532.115,154.038 537.457,141.673 542.799,128.747 548.142,115.236 553.484,101.117 558.827,86.3618 564.169,70.9436 569.511,54.8333 574.854,38.0005 580.196,20.413 
 
 "/>
 
 
 
 
 
 
 
 y1 
 
 
 
 y2

In [136]:
[f(2) for f in F]

4-element Array{Any,1}:
 1      
 2      
 4      
 7.38906

In [137]:
for f in F
    fx = f(1.2)
    println("f(1.2) = $fx")
end

f(1.2) = 1
f(1.2) = 1.2
f(1.2) = 1.44
f(1.2) = 3.3201169227365472


## Exercício para 10 de Outubro

Muitas vezes, além dos erros obtidos numa medição, existem os
**outliers**, isto é, dados que foram obtidos de *maneira errada*.
Talvez por algum instrumento defeituoso, ou alguma metodologia
que foi entendida errada.

In [143]:
m = 500
x = sort(5 * rand(m) - 2) # x ∈ [-2,3]
y = 2 - 0.5 * x + 0.3 * x.^2 + rand(m)*0.1 + [rand() < 0.05 ? 2 : 0 for i=1:m]

scatter(x, y, ms=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [144]:
A = [ones(x) x x.^2];
α = A\y
scatter(x, y, ms=3)
plot!(x->dot(α,[1,x,x^2]), x[1], x[end], c=:red, lw=2)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 51.3,157.767 56.6423,164.668 61.9847,171.446 67.3271,178.099 72.6695,184.629 78.0119,191.035 83.3543,197.317 88.6967,203.475 94.039,209.509 99.3814,215.419 
 104.724,221.206 110.066,226.868 115.409,232.407 120.751,237.822 126.093,243.113 131.436,248.28 136.778,253.323 142.121,258.242 147.463,263.038 152.805,267.709 
 158.148,272.257 163.49,276.681 168.832,280.981 174.175,285.157 179.517,289.209 184.86,293.138 190.202,296.942 195.544,300.623 200.887,304.18 206.229,307.613 
 211.572,310.922 216.914,314.107 222.256,317.168 227.599,320.106 232.941,322.919 238.283,325.609 243.626,328.175 248.968,330.617 254.311,332.935 259.653,335.129 
 264.995,337.199 270.338,339.146 275.68,340.968 281.023,342.667 286.365,344.242 291.707,345.693 297.05,347.02 302.392,348.223 307.734,349.303 313.077,350.258 
 318.419,351.09 323.762,351.798 329.104,352.382 334.446,352.842 339.789,353.178 345.131,353.39 350.474,353.479 355.816,353.443 361.158,353.284 366.501,353.001 
 371.843,352.594 377.185,352.063 382.528,351.408 387.87,350.629 393.213,349.727 398.555,348.7 403.897,347.55 409.24,346.276 414.582,344.878 419.925,343.356 
 425.267,341.71 430.609,339.941 435.952,338.047 441.294,336.03 446.636,333.889 451.979,331.624 457.321,329.235 462.664,326.722 468.006,324.085 473.348,321.325 
 478.691,318.44 484.033,315.432 489.376,312.3 494.718,309.044 500.06,305.664 505.403,302.16 510.745,298.532 516.087,294.781 521.43,290.905 526.772,286.906 
 532.115,282.783 537.457,278.536 542.799,274.165 548.142,269.67 553.484,265.052 558.827,260.309 564.169,255.443 569.511,250.453 574.854,245.339 580.196,240.101 
 
 "/>
 
 
 
 
 
 
 
 y1 
 
 
 
 y2

Estude alguma maneira de remover outliers de um conjunto de dados,
faça uma explicação dessa maneira, e faça um código que faz
essa remoção de outliers de um conjunto de dados. Depois, teste
com o problema dado.

Os passos são:
- resolva com outliers e plote
- remova outliers com o seu código
- resolva sem outliers e plote